# Section II: Some fundamentals


In [ ]:
# standard imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import scipy.special
import scipy.stats as stats
# force plots to appear inline on this page
%matplotlib inline

## Overfitting and regularisation
### Overfitting 
### Penalisation
#### Lasso / ridge regression
### Randomised regularisation
#### Bootstrap / bagging

## Probabilistic approaches
#### Random variables
A *random variable* is a variable that can take on different values; i.e. one that is "unassigned". Proability theory allows us to manipulate random variables without having to assign them a specific value.

#### Distributions
A *probability* distribution defines how likely different states of a random variable are. The probability distribution of a random variable $x$ is written:
$$P(x)$$
Random variables can be continuous (e.g. the height of a person) or discrete (the value showing on the face of a dice). The distribution of a discrete variable is described with a *probability mass function* (PMF) which gives each outcome a specific value. A continuous variable has a *probability density function* (PDF) which specifies the spread of the probability as a continuous function.

A probability distribution must assign probabilities in the range 0 (impossible) to 1 (definite) and the PMF or PDF **must** integrate to exactly 1 as the random variable under consideration must take on *some* value. Note that a **PDF** can have a value >1 if the integral of the function is still = 1. A PMF always has $P(x)<1 \forall x$


#### Advantages of probabilistic modelling
#### Disadvantages of probabilistic modelling

In [ ]:
# the PMF of the sum of two dice rolls
def two_dice():
    # form the sum of the cross product of these possibilities
    roll_two = [i+j for i in range(6) for j in range(6)]
    # now plot the histogram
    plt.hist(roll_two, normed=True, bins=range(12))
    plt.xlabel("Sum of rolls x")
    plt.ylabel("P(x)")


In [ ]:
two_dice()

In [ ]:
# Plot the PDF of the normal distribution
def plot_normal():
    # plot the normal (Gaussian distibution) along with a set of points drawn from that distribution
    x = np.linspace(-4,4,100)
    y = stats.norm.pdf(x) # mean 0, std. dev. 1
    plt.plot(x,y)
    plt.axhline(0, color='k', linewidth=0.2) # axis line
 
    # mark the mean
    plt.text(0, 0.51, '$\mu$')
    plt.axvline(0, color='r')
    # highlight one std. dev. to the right
    plt.axvspan(0,1, facecolor='b', alpha=0.1)
    plt.text(1.2, 0.3, '$\sigma$')
    # take 1000 random samples and scatter plot them
    samples = stats.norm.rvs(0,1,1000)
    plt.scatter(samples, np.full(samples.shape, .2), s=4, c='b', alpha=0.03)
    plt.xlabel("$x$")
    plt.ylabel("$P(x)$")
    

In [ ]:
plot_normal()

### Joint, marginal, conditional
The *joint probability* of two random variables is written $$P(x,y)$$ and gives the probability that $x$ and $y$ take the same value simultaneously. The *marginal probability* is the derivation of $P(x)$ from $P(x,y)$ by integrating (summing) over all the possible choices of $y$:
$$P(x) = \int P(x,y) dy$$

Two random variables are *independent* if the they do not have any dependence on each other. If this is the case then the joint distribution is just the product of the individual distributions:
$p(x,y) = p(x)p(y)$

The *conditional probability* of $x$ **given** $y$ is written as $$p(x|y)$$ and can be computed as $$p(x|y) = \frac{p(x,y)}{p(x)}$. This tells us how likely $x$ is to occur if we already know  (or fix) the value of $y$.

The *expected value* of a random variable with respect to a function is the function weighted by the probability at each point:
$$E[f(x)] = \int p(x)f(x) dx$$
or
$$E[f(x)] = \sum_{x} p(x)f(x)$$ in the discrete case. 


These concepts are easier to see visually.


In [ ]:
def joint_marginal(cov):
    # create an independent 2D normal distribution
    x,y = np.meshgrid(np.linspace(-3,3,50), np.linspace(-3,3,50))
    pos = np.empty(x.shape + (2,))
    pos[:,:,0] = x
    pos[:,:,1] = y
    joint_pdf = stats.multivariate_normal.pdf(pos, [0,0], cov)
    fig = plt.figure()
    # plot the joint
    ax = fig.add_subplot(2,2,1)
    ax.axis('equal')
    ax.pcolor(x,y,joint_pdf, cmap='gist_heat')
    # plot the marginals
    ax = fig.add_subplot(2,2,3)
    ax.axis('equal')
    
    ax.plot(x[0,:], np.sum(joint_pdf, axis=0))
    ax = fig.add_subplot(2,2,2)
    ax.axis('equal')
    
    ax.plot(np.sum(joint_pdf, axis=1), x[0,:])
    # plot p(x|y)
    ax = fig.add_subplot(2,2,4)
    ax.axis('equal')
    
    marginal = np.tile(np.sum(joint_pdf, axis=0), (joint_pdf.shape[0],1))
    ax.pcolor(x,y,joint_pdf/marginal, cmap='gist_heat')
    
    



In [ ]:
# x and y indepependent
joint_marginal([[1,0], [0,1]])


In [ ]:
# x and y covary
joint_marginal([[0.5,0.5], [0.5,1]])

We can draw samples from a distribution, which gives us a set of definite (non-random) variables which are distributed according to the PDF or PMF. The mean $\mu$ of a set of samples from a distribution is an estimate of the expectation, which improves as the number of samples $n$ increases. If we apply a function $f(x)$ to the samples, the arithmetic mean is an estimate of $E[f(x)]$.




## Normal distribution
The normal distribution (above) is very widely used as a model of random variables. It is fully specified by its mean $\mu$ and the covariance matrix $\Sigma$. If you imagine the normal distribution to be a ball shaped mass in space, the mean *translates* the mass, and covariance applies a transformation matrix (scale, rotate and shear) to the ball. [The exact transformation matrix which does this transform is $\Sigma^{-1/2}$].

#### Normal modelling
It seems that this might be a very limiting choice but there are two good reasons for this:
1. Normal variables have very nice mathematical properties and are easy to work with analyitically (i.e. without relying on numerical computation).
2. The *central limit theory* tells us that any sum of random variables (however they are distributed) will tend to a *Levy stable distribution* as the number of variables being summed increases. For most random variables encountered, this means the normal distribution (one specific Levy stable distribution).

#### Tails
Obviously, normal variables cannot model multi-modal distributions well (where the distribution has multiple "spikes"), although mixtures of normal variables can be used to model these. The main issue with normal approximations is that the *tails* (the low probability areas at the extremes of the function) are very "light" -- the normal distribution gives very little probability to rare events. Other distributions, such as the *t-distribution* can model phenomena where there are significant rare events. 



In [ ]:
def clt():
    # demonstrate the central limit theorem
    for i in range(8):
        x = np.zeros((800,))
        # add i copies of samples drawn from uniform (flat) distribution together
        for j in range(i+1):
            x += np.random.uniform(-0.5,0.5, x.shape)
        plt.figure()
        plt.hist(x, bins=np.linspace(-4,4,40), normed=True)
        plt.xlabel("$x$")
        plt.ylabel("$P(x)$")

In [ ]:
clt()

In [ ]:
# Compare normal and t-distributed values
def plot_normal_and_t():
    x = np.linspace(-8, 8,100)
    plt.figure()
    plt.plot(x, stats.norm.pdf(x))
    plt.plot(x, stats.t.pdf(x,1)) # t-distribution with degrees-of-freedom = 1
    # fill the difference in the tails
    plt.fill_between(x, stats.t.pdf(x,1), stats.norm.pdf(x), where=stats.t.pdf(x,1)>stats.norm.pdf(x), alpha=0.1)
    plt.xlabel("$x$")
    plt.ylabel("$p(x)$")

    plt.figure()
    plt.plot(x, stats.norm.logpdf(x))
    plt.plot(x, stats.t.logpdf(x,1)) # t-distribution with degrees-of-freedom = 1
    plt.xlabel("$x$")
    plt.ylabel("$\log p(x)$")
    
plot_normal_and_t()

## Probability theory
#### Probability as a calculus of belief
#### Axioms of probability
#### Bayes' Rule
#### Prior, likelihood, posterior
#### Integration over the evidence
##### Monte Carlo approximations
### Example: language modelling
#### n-gram character models
#### Bigram matrix
#### Sampling from the model
#### Evaluating likelihood

## Information theory
### Entropy
#### Shannon's law
#### Entropy examples
#### Mutual information
### Example: Fitts' law as an information theoretic model

## High-dimensional spaces
### What is a high-dimensional space?

### Dealing with high-D problems
#### Pre-training
#### Heavier tailed distributions


The volume of a n-D sphere with radius $1/2$  is $$ V_n(R) = \frac{\pi^{n/2}}{\Gamma({n/2}+1)}\frac{1}{2}^n$$ (i.e. inscribed in a hypersphere). The volume of a unit cube is $$1^n=1$$

In [ ]:
def sphere_volume(n):
    return 0.5**n * np.pi**(n/2.0) / scipy.special.gamma(n/2.0+1)

In [ ]:
x = np.arange(0,20)
plt.plot(x, [sphere_volume(xi) for xi in x])
plt.xlabel("Dimension")
plt.ylabel("Volume")
plt.figure()
plt.semilogy(x, [sphere_volume(xi) for xi in x])
plt.xlabel("Dimension")
plt.ylabel("Volume")

We can generate points randomly in a hypersphere. It's hard to visualise the hypersphere, but we can show the radii of points on a 2D circle:

In [ ]:
def sphere_points(n, d):
    # generate points on the unit circle (uniformly)    
    xn = np.random.normal(0,1,(n,2))
    r = np.sqrt(np.sum(xn**2, axis=1))    
    surface_points = (xn.T/r).T
    
    # generate points on the unit d-dimensional hypershphere (uniformly)    
    xv = np.random.normal(0,1,(n,d))
    r_d = np.sqrt(np.sum(xv**2, axis=1))    
    d_surface_points = (xv.T/r_d).T
    
    # generate points on the unit line
    xt = np.random.normal(0,1,(n,1))
                
    # radii of points uniformly distributed in a n-d hypersphere
    # can be drawn by sampling using the formula below
    # [see: http://math.stackexchange.com/questions/87230/picking-random-points-in-the-volume-of-sphere-with-uniform-probability?rq=1 ]
    radius = np.random.uniform(0, 1, n) ** (1.0/d) * 0.5
    return (surface_points.T*radius).T, radius, (d_surface_points.T*radius).T

We can define a function to plot this. We'll show the 1D distribution of radii, the distribution of radii as if they were on a circle and the true 2D projection of the hypersphere points.

In [ ]:
def plot_sphere_density(d):
    sphere_pts, line_pts, hyp_pts = sphere_points(2000,d)
    
    # plot the 2D sphere at the corresponding radius
    plt.scatter(sphere_pts[:,0], sphere_pts[:,1], alpha=0.5, s=2)    
    # plot 1D points at the corresponding radius
    plt.scatter(line_pts, np.zeros_like(line_pts), c='g', alpha=0.1, s=2)    
    # plot the 2D projection of the hypersphere points
    plt.scatter(-line_pts, np.zeros_like(line_pts), c='g', alpha=0.1, s=2)
    if d>1:
        plt.scatter(hyp_pts[:,0], hyp_pts[:,1], c='r', alpha=0.5, s=2)
    plt.axis("equal")

In [ ]:
plot_sphere_density(1)

In [ ]:
plot_sphere_density(2)

In [ ]:
plot_sphere_density(3)

In [ ]:
plot_sphere_density(8)

In [ ]:
plot_sphere_density(128)

#### Conclusion
If you look at any 2D slice of a hypersphere, the points will appear to be concentrated at the centre as the dimension increases. But almost all of the volume is actually in a very thin shell at the outside of the sphere. This is quite counter-intuitive.

## Kernel trick: a useful high-d space
### Kernel basics
### Linear decisions in kernel space
### Kernel functions
### Non-real data: bag-of-words


## Dealing with time-series
### Windowing
### Delay embedding
### Derivative information

### Stateful algortihms
#### Hidden markov model
#### Kalman filter
#### Particle filter